In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import json
from scipy.stats import gaussian_kde
from scipy.spatial.distance import jensenshannon
from statsmodels.stats.proportion import proportions_ztest
import gradio as gr


c:\Users\totog\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Chargement des données
df_data = pd.read_csv("../Data/Etude sur les Français et l_Information/les-francais-et-l-information-propre.csv")

# Chargement des questions
with open("../Data/Etude sur les Français et l_Information/questionnaire.json", "r") as f:
    questionnaire = json.load(f)


C:\Users\totog\AppData\Local\Temp\ipykernel_16492\1700436274.py:2: DtypeWarning: Columns (16,20,104,105,111,112,113,120,121,128,129,136,137,145,152,153,234,264,265,279,280,281,282,296,305,316,317,318,330,342,343,344) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv("../Data/Etude sur les Français et l_Information/les-francais-et-l-information-propre.csv")


## Orientations politiques selon les chaines

On charge les données, et on les nettoie.

In [3]:
# Variables de filtres
liste_col_X = [col for col in df_data.columns if col.startswith("SOURCES1DR_B4_R") or col.startswith("SOURCES1ER_B4_R")]
X_brut = df_data[liste_col_X]

# Variables à observer
liste_col_Y =  ["NOU1_R"] + ["PP3_R"] + [col for col in df_data.columns if col.startswith("VA1_R") ]
Y = df_data[liste_col_Y]

## Supression des nan

print("Avant suppression des nan: ")
print("Taille de X: ", X_brut.shape)
print("Taille de Y: ", Y.shape)

condition_X = X_brut.isna().all(axis=1)
condition_Y = Y.isna().any(axis=1)
indices_nan = df_data.index[condition_X | condition_Y]
X_brut = X_brut.drop(index = indices_nan)
Y = Y.drop(index = indices_nan)

print("\nAprès suppression des nan: ")
print("Taille de X: ", X_brut.shape)
print("Taille de Y: ", Y.shape)

## Traitement de X

# Liste des chaîne de TV
liste_valeurs_avec_nan= pd.unique(X_brut.values.ravel())
liste_valeurs = [val for val in liste_valeurs_avec_nan if not pd.isna(val)]

# Initialiser un DataFrame avec une colonne par chaîne et des zéros
X = pd.DataFrame(0, index=X_brut.index, columns=liste_valeurs)

# Remplir le DataFrame final
for i, row in X_brut.iterrows():
    # Extraire les réponses non NaN de la ligne courante
    reponses = row.dropna().unique().tolist()
    # Mettre 1 dans les colonnes correspondantes aux réponses
    X.loc[i, reponses] = 1

# Suppression de la colonne donnant une information redondante
X = X.drop(columns=['Rien de tout cela'])

## Traitement de Y

Y["NOU1_R"] = 2 * (Y["NOU1_R"] - Y["NOU1_R"].min()) / (Y["NOU1_R"].max() - Y["NOU1_R"].min()) - 1
Y["PP3_R"] = Y["PP3_R"].apply(lambda x: x.replace("Je ne suis pas inscrit sur les listes électorales", "pas inscrit·e"))
Y["PP3_R"] = Y["PP3_R"].apply(lambda x: x.replace("Je ne suis pas inscrit·e sur les listes électorales", "pas inscrit·e"))
Y["PP3_R"] = Y["PP3_R"].apply(lambda x: x.replace('Je me suis abstenu', "abstenu·e"))
Y["PP3_R"] = Y["PP3_R"].apply(lambda x: x.replace("J'ai voté blanc ou nul", "vote blanc / nul"))
Y["PP3_R"] = Y["PP3_R"].apply(lambda x: x.replace('Autre Gauche, Nathalie Arthaud ou Philppe Poutou ', 'Nathalie Arthaud / Philppe Poutou'))

# Liste des candidats par ordre de gauche à droite
candidats = [
    "Nathalie Arthaud / Philppe Poutou",
    "Fabien Roussel",
    "Jean Luc Mélenchon",
    "Anne Hidalgo",
    "Yannick Jadot",
    "Emmanuel Macron",
    "Jean Lassalle",
    "Valérie Pécresse",
    "Nicolas Dupont Aignan",
    "Marine Le Pen",
    "Éric Zemmour",
    "vote blanc / nul",
    "abstenu·e",
    'pas inscrit·e',
    'Non réponse'
]

# Réponses possibles sur les sujets clivants
responses = ['Tout à fait', 'Plutôt', 'Ne sait pas', 'Plutôt pas', 'Pas du tout']

# résultats des présidentielles 2022 au premier tour

premier_tour = {
    "Nathalie Arthaud / Philppe Poutou": 0.95,
    "Fabien Roussel": 1.65,
    "Jean Luc Mélenchon": 15.83,
    "Anne Hidalgo":1.26,
    "Yannick Jadot": 3.34,
    "Emmanuel Macron": 20.07,
    "Jean Lassalle": 2.26,
    "Valérie Pécresse": 3.45,
    "Nicolas Dupont Aignan": 1.49,
    "Marine Le Pen": 16.69,
    "Éric Zemmour": 5.1,
    "abstenu·e": 25.14,
    "vote blanc / nul": 2.3,
    'pas inscrit·e':0,
    'Non réponse':0
}


Avant suppression des nan: 
Taille de X:  (3346, 22)
Taille de Y:  (3346, 10)

Après suppression des nan: 
Taille de X:  (2821, 22)
Taille de Y:  (2821, 10)


Les 4 fonctions ci-dessous tracent les graphiques du dashboard

In [14]:
def generate_kde(filtered_Y: pd.DataFrame, non_filtered_Y: pd.DataFrame, show_complementary: bool) -> go.Figure:
    
    # Calculer les densités pour les KDE plots
    kde_filtered = gaussian_kde(filtered_Y["NOU1_R"])
    kde_non_filtered = gaussian_kde(non_filtered_Y["NOU1_R"])

    # Créer la gamme de valeurs pour les courbes KDE
    x_vals = np.linspace(min(filtered_Y["NOU1_R"].min(), non_filtered_Y["NOU1_R"].min()),
                         max(filtered_Y["NOU1_R"].max(), non_filtered_Y["NOU1_R"].max()), 500)
    
    # Calculer les valeurs des distributions pour comparaison
    pdf_filtered = kde_filtered(x_vals)
    pdf_non_filtered = kde_non_filtered(x_vals)

    # Normaliser les distributions (somme des densités = 1)
    pdf_filtered /= np.sum(pdf_filtered)
    pdf_non_filtered /= np.sum(pdf_non_filtered)

    # Calculer la distance de Jensen-Shannon
    js_distance = jensenshannon(pdf_filtered, pdf_non_filtered)

    # Moyennes des orientations politiques
    mean_filtered = filtered_Y["NOU1_R"].mean()
    mean_non_filtered = non_filtered_Y["NOU1_R"].mean()

    # Tracer la courbe de densité
    fig_kde = go.Figure()
    fig_kde.add_trace(go.Scatter(
        x=x_vals, y=pdf_filtered, mode='lines', fill='tozeroy',
        name=f'Individus filtrés ({len(filtered_Y)})',
        fillcolor='rgba(255, 165, 0, 0.6)', line=dict(color='orange')
    ))
    fig_kde.add_trace(go.Scatter(
        x=x_vals, y=pdf_non_filtered, mode='lines', fill='tozeroy',
        name=f'{"Complémentaire" if show_complementary else "Toute la population"} ({len(non_filtered_Y)})',
        fillcolor='rgba(0, 204, 105, 0.6)', line=dict(color='#00CC96')
    ))

    # Ajouter les lignes verticales pour les moyennes
    fig_kde.add_shape(
        type="line",
        x0=mean_filtered,
        x1=mean_filtered,
        y0=0,
        y1=1,
        xref="x",
        yref="paper",
        line=dict(color="orange", dash="dash"),
        name="Moyenne filtrés"
    )
    fig_kde.add_shape(
        type="line",
        x0=mean_non_filtered,
        x1=mean_non_filtered,
        y0=0,
        y1=1,
        xref="x",
        yref="paper",
        line=dict(color="green", dash="dash"),
        name="Moyenne complémentaire ou population"
    )

    # Ajouter une annotation indiquant la divergence entre les distributions
    annotation_text = f"Distance de Jensen-Shannon : {js_distance * 100:.2f}%"
    fig_kde.add_annotation(
        text=annotation_text,
        xref="paper", yref="paper",
        x=0.5, y=-0.25, showarrow=False,
        font=dict(size=12), align="center"
    )

    # Mise à jour du layout pour KDE plot
    fig_kde.update_layout(
        title="Distribution de l'orientation politique",
        xaxis_title="Orientation politique",
        yaxis_title="Densité",
        showlegend=True,
        xaxis=dict(
            tickvals=[-1, 0, 1],
            ticktext=['Gauche', 'Centre', 'Droite']
        )
    )

    return fig_kde



def generate_barplot(filtered_Y: pd.DataFrame) -> go.Figure:

    # Données pour les individus filtrés
    filtered_counts = filtered_Y["PP3_R"].value_counts(normalize=False).reindex(candidats, fill_value=0)
    filtered_percentages = (filtered_counts / filtered_counts.sum() * 100).round(2)

    # Création du DataFrame pour les deux séries de données
    barplot_data = pd.DataFrame({
        'Candidat': candidats,
        'Individus filtrés (%)': filtered_percentages.values,
        'Résultats officiels (%)': [premier_tour[candidat] if candidat in premier_tour else 0 for candidat in candidats]
    })

    # Création du barplot avec des barres groupées
    barplot_fig = px.bar(
        barplot_data.melt(id_vars='Candidat', var_name='Source', value_name='Pourcentage'),
        x='Candidat',
        y='Pourcentage',
        color='Source',
        barmode='group',
        title="Comparaison des votes filtrés et des résultats officiels"
    )

    # Mise à jour des axes
    barplot_fig.update_layout(
        xaxis_title="Candidat",
        yaxis_title="Pourcentage",
        xaxis=dict(categoryorder="array", categoryarray=candidats, tickangle=45),
        yaxis=dict(ticksuffix="%")
    )

    return barplot_fig


def generate_questions(filtered_Y: pd.DataFrame) -> go.Figure:
    # Liste des réponses dans l'ordre du dégradé
    responses = ["Tout à fait", "Plutôt", "Je ne sais pas", "Plutôt pas", "Pas du tout"]

    # Couleurs associées à chaque réponse (du bleu au rouge)
    colors = {
        "Tout à fait": "rgb(0, 0, 255)",      # Bleu
        "Plutôt": "rgb(100, 149, 237)",      # Bleu clair
        "Je ne sais pas": "rgb(211, 211, 211)",  # Gris
        "Plutôt pas": "rgb(255, 165, 0)",    # Orange
        "Pas du tout": "rgb(255, 0, 0)"      # Rouge
    }

    # Initialiser les données pour les barres empilées
    question_data = {response: [] for response in responses}
    question_labels = []

    for question in filtered_Y.columns[2:]:
        # Calculer les pourcentages des réponses pour chaque question
        counts_filtered = filtered_Y[question].value_counts(normalize=True) * 100

        # Ajouter les données pour la figure (pour les barres empilées)
        for response in responses:
            question_data[response].append(counts_filtered.get(response, 0))  # Ajouter 0% si une réponse est absente
        question_labels.append(questionnaire[question])  # Ajouter l'intitulé de la question

    # Créer une figure avec barres empilées horizontales
    fig_questions = go.Figure()

    for response in responses:
        fig_questions.add_trace(go.Bar(
            y=question_labels,  # Les questions sur l'axe Y
            x=question_data[response],  # Les pourcentages sur l'axe X
            name=response,
            orientation='h',
            marker=dict(color=colors[response]),  # Appliquer la couleur associée
            hovertemplate='%{x:.2f}%<extra></extra>'  # Affiche le pourcentage avec 2 décimales et supprime les infos supplémentaires
        ))

    # Mettre à jour le layout de la figure
    fig_questions.update_layout(
        barmode='stack',  # Empiler les barres
        title="Répartition des opinions sur les sujets clivants (individus filtrés)",
        xaxis_title="Pourcentage des réponses",
        yaxis_title="Question",
        legend_title="Réponses"
    )

    return fig_questions

def generate_difference(filtered_Y: pd.DataFrame, non_filtered_Y: pd.DataFrame) -> go.Figure:

    question_labels = []  # Liste des questions (intitulés)
    differences = []  # Différences entre les pourcentages
    significative = []  # Liste des résultats de tests de significativité

    for question in filtered_Y.columns[2:]:
        # Calculer les effectifs pour les réponses "Tout à fait" ou "Plutôt" dans les deux groupes
        agree_filtered = filtered_Y[question].isin(["Tout à fait", "Plutôt"]).sum()
        agree_non_filtered = non_filtered_Y[question].isin(["Tout à fait", "Plutôt"]).sum()

        total_filtered = len(filtered_Y[question])
        total_non_filtered = len(non_filtered_Y[question])

        # Calculer les pourcentages
        percentage_filtered = (agree_filtered / total_filtered) * 100
        percentage_non_filtered = (agree_non_filtered / total_non_filtered) * 100

        # Différence entre les pourcentages
        diff = percentage_filtered - percentage_non_filtered
        differences.append(diff)

        # Test d'égalité des proportions
        count = np.array([agree_filtered, agree_non_filtered])
        nobs = np.array([total_filtered, total_non_filtered])
        stat, p_value = proportions_ztest(count, nobs)

        # Tester si la différence est significative (alpha = 0.05)
        significative.append(p_value < 0.05)

        # Ajouter l'intitulé de la question
        question_labels.append(questionnaire[question])

    # Couleur des barres : gris si non significatif, orange sinon
    colors = ['#00CC96' if sig else 'lightgray' for sig in significative]

    # Créer la figure
    fig_diff = go.Figure()
    fig_diff.add_trace(go.Bar(
        y=question_labels,  # Intitulés des questions sur l'axe Y
        x=differences,  # Différences sur l'axe X
        orientation='h',  # Barres horizontales
        marker_color=colors,  # Couleur des barres
        hovertemplate='<b>%{y}</b><br>Différence : %{x:.2f}%<br>'
    ))

    # Mettre à jour le layout
    fig_diff.update_layout(
        title="Différences entre la part de personnes en accord avec l'affirmation entre les groupes filtrés et non filtrés",
        xaxis_title="Différence (%) (Filtrés - Non filtrés)",
        yaxis_title="Questions",
        xaxis=dict(
            zeroline=True,  # Afficher une ligne à X=0
            zerolinecolor="black",
            zerolinewidth=1
        ),
        showlegend=False  # Pas de légende
    )

    return fig_diff


On peut mantenant générer notre dashboard.

In [15]:
# Fonction pour filtrer les données et générer les graphiques
def generate_dashboard(filter_include, filter_exclude, show_complementary):
    include_cols = filter_include  # Chaînes sélectionnées pour "inclus"
    exclude_cols = filter_exclude  # Chaînes sélectionnées pour "exclus"

    # Filtrer les individus pour X
    mask_include = np.logical_or.reduce([X[col] == 1 for col in include_cols]) if include_cols else np.ones(len(X), dtype=bool)
    mask_exclude = np.logical_and.reduce([X[col] == 0 for col in exclude_cols]) if exclude_cols else np.ones(len(X), dtype=bool)
    mask = mask_include & mask_exclude

    # Filtrer Y
    filtered_Y = Y.loc[mask]
    if show_complementary:
        non_filtered_Y = Y.loc[~mask]  # Complémentaire des individus filtrés
    else:
        non_filtered_Y = Y  # Toute la population

    ## 1 - kde plot pour les orientations politiques
    fig_kde = generate_kde(filtered_Y, non_filtered_Y, show_complementary)

    ## 2 - barplot des votes
    barplot_fig = generate_barplot(filtered_Y)

    ## 3 - barplot opinion sur des sujets clivants (barres horizontales)
    fig_questions = generate_questions(filtered_Y)

    ## 4 - Différence d'opinion entre les individus filtrés et non filtrés
    fig_diff = generate_difference(filtered_Y, non_filtered_Y)

    return fig_kde, barplot_fig, fig_questions, fig_diff

# Interface Gradio mise à jour
def dashboard_interface():
    with gr.Blocks() as interface:
        gr.Markdown("## Dashboard des chaînes TV et orientation politique")
        gr.Markdown("Sélectionnez les chaînes que les individus doivent ou ne doivent pas regarder pour filtrer les données.")

        with gr.Row():
            include = gr.CheckboxGroup(label="Chaînes à inclure (regardées)", choices=list(X.columns), value=[])
            exclude = gr.CheckboxGroup(label="Chaînes à exclure (non regardées)", choices=list(X.columns), value=[])

        with gr.Row():
            show_complementary = gr.Checkbox(label="Comparer au complémentaire des individus filtrés (si non, compare à l'ensemble de la population)", value=True)

        button = gr.Button("Mettre à jour")

        with gr.Row():
            plot_kde = gr.Plot(label="Distribution de l'orientation politique")
            plot_bar = gr.Plot(label="Nombre de voix par candidat")
        with gr.Row():
            plot_questions = gr.Plot(label="Répartition des opinions par question")
        with gr.Row():
            plot_diff = gr.Plot(label="Différences d'opinions")

        # Relier le bouton à la fonction
        button.click(
            fn=generate_dashboard,
            inputs=[include, exclude, show_complementary],
            outputs=[plot_kde, plot_bar, plot_questions, plot_diff]
        )

    return interface


# Lancer le dashboard
dashboard_interface().launch(share=True)

* Running on local URL:  http://127.0.0.1:7884
* Running on public URL: https://b21bdb1283c94a903a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


c:\Users\totog\anaconda3\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

c:\Users\totog\anaconda3\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

c:\Users\totog\anaconda3\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

Traceback (most recent call last):
  File "c:\Users\totog\anaconda3\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api